# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [28]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [29]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,esperance,-33.8667,121.9000,17.47,70,0,3.80,AU,1681339618
1,1,acapulco,16.8634,-99.8901,30.90,62,0,5.14,MX,1681339609
2,2,hilo,19.7297,-155.0900,25.85,75,75,4.63,US,1681339620
3,3,fortuna,40.5982,-124.1573,11.19,61,11,4.92,US,1681339621
4,4,port alfred,-33.5906,26.8910,18.86,63,100,3.55,ZA,1681339621


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [35]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",frame_width = 700,
    frame_height = 500, size = "Humidity",
    scale = 0.7,
    color = "City")

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [31]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_cities = city_data_df[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)
                                    & (city_data_df["Cloudiness"] == 0) & (city_data_df["Wind Speed"] < 4.5)] 


# Drop any rows with null values
ideal_weather_cities.dropna()


# Display sample data
ideal_weather_cities.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
200,200,wilmington,34.2257,-77.9447,22.95,32,0,2.24,US,1681339626
327,327,sundargarh,22.1167,84.0333,25.25,19,0,0.50,IN,1681339878
339,339,tiznit,29.5833,-9.5000,21.27,26,0,1.33,MA,1681339888
447,447,saint-pierre,-21.3393,55.4781,24.20,78,0,3.09,RE,1681339968
494,494,chatra,24.2167,84.8667,24.01,21,0,2.66,IN,1681340007


### Step 3: Create a new DataFrame called `hotel_df`.

In [32]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df["Hotel Name"] = ""


# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
200,wilmington,US,34.2257,-77.9447,32,
327,sundargarh,IN,22.1167,84.0333,19,
339,tiznit,MA,29.5833,-9.5000,26,
447,saint-pierre,RE,-21.3393,55.4781,78,
494,chatra,IN,24.2167,84.8667,21,
507,bella vista,US,36.4295,-94.2316,32,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [33]:
# Set parameters to search for a hotel
radius = 10000
# latitude = []
# longitude = []
categories = "accommodation.hotel"
limit = 20

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key    
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wilmington - nearest hotel: Hell's Kitchen
sundargarh - nearest hotel: No hotel found
tiznit - nearest hotel: No hotel found
saint-pierre - nearest hotel: Tropic Hotel
chatra - nearest hotel: No hotel found
bella vista - nearest hotel: 21c Museum Hotel Bentonville


,City,Country,Lat,Lng,Humidity,Hotel Name
200,wilmington,US,34.2257,-77.9447,32,Hell's Kitchen
327,sundargarh,IN,22.1167,84.0333,19,No hotel found
339,tiznit,MA,29.5833,-9.5000,26,No hotel found
447,saint-pierre,RE,-21.3393,55.4781,78,Tropic Hotel
494,chatra,IN,24.2167,84.8667,21,No hotel found
507,bella vista,US,36.4295,-94.2316,32,21c Museum Hotel Bentonville


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [34]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",frame_width = 800,
    frame_height = 600, scale =1,size = "Humidity",
    hover_cols = ["City", "Hotel Name", "Country"], color= "City")

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)